In [ ]:
import pandas as pd
import numpy as np
import re
import time

from bs4 import BeautifulSoup
import requests

import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import asyncio
import aiohttp
import json

Scraping dal sito RFI.

In [ ]:
url = "https://www.rfi.it/it/stazioni.html"

def browser():
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    return driver
    time.sleep(3)

def main():
    driver = browser()
    driver.find_element(By.ID, "onetrust-reject-all-handler").click()
    
    time.sleep(10)
    
    stazioni = []

    for i in range(259):
        elements = driver.find_elements(By.CLASS_NAME, "stazione")

        for element in elements:
            print(element.text)
            stazioni.append(element.text)

        WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn-fs.has-icon-right.icon--chevron-right.pagination--next-btn"))).click()

    elements = driver.find_elements(By.CLASS_NAME, "stazione")

    for element in elements:
        print(element.text)
        stazioni.append(element.text)

    time.sleep(10)
    print(stazioni)
    return stazioni

In [ ]:
stazioni = main()

In [ ]:
for i in range(len(stazioni)):
  stazioni[i] = stazioni[i].lower()
  stazioni[i] = stazioni[i].replace(' ', '-')
  stazioni[i] = stazioni[i].replace('.', '-')

In [ ]:
siti = []
for stazione in stazioni:
    siti.append("https://www.rfi.it/it/stazioni/" + stazione + ".html")

staz_siti = list(zip(stazioni, siti))

stazioni_binari= {}

In [ ]:
async def get(url, stazione, session):
    try:
        async with session.get(url=url) as response:
            resp = await response.text()
            resp = resp.split("Questa stazione è dotata complessivamente di *")
            numero_binari = resp[1].split("binari*")[0]
            print(numero_binari)

            stazioni_binari[stazione]= numero_binari
            print(stazione, ':', stazioni_binari[stazione])

            print("Successfully got url {} with resp of length {}.".format(url, len(resp[1])))
    except Exception as e:
        print("Unable to get url {} due to {}.".format(url, e.__class__))

async def main(urls):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[get(url, stazione, session) for stazione, url in urls])
    print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))

In [ ]:
await main(staz_siti)

In [ ]:
with open('stazioni_binari.json', 'w') as outfile:
    json.dump(stazioni_binari, outfile)